In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import pandas as pd

In [ ]:
#df1 = pd.read_csv("small-2oq-c1r.csv", error_bad_lines=False, low_memory=False)
#print(df1)

#df2 = df1.set_index('categories', drop=False)
#df3 = df2.loc["Apparels>Women>Western Wear>Shirts, Tops & Tunics>Tops"]
#df3.to_csv('tops_listings_small.csv', index=False)

In [ ]:
df3 = pd.read_csv("tops_listings_small.csv")
df3.shape


In [ ]:
def getImageUrls(prod):
    url_str = prod.imageUrlStr
    img_urls = url_str.split(';')
    return img_urls

In [ ]:
import cv2
import urllib.request
import numpy as np

def getImagesFromUrls(urls):
    for url in urls:
        req = urllib.request.urlopen(url)
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        img = cv2.imdecode(arr, -1) # 'Load it as it is'
        print(img.shape)
        
    return img


In [ ]:
urls = getImageUrls(df3.iloc[0])

print("total images in this listing are :", len(urls))
urls[0]

image = getImagesFromUrls(urls)
print(image.shape)

In [ ]:
cv2.imshow('image', image)

In [ ]:
i=0
df3.iloc[i]
df3.iloc[53].description

In [ ]:
def isDuplicate(prod1, prod2):
    if(prod1.productId == prod2.productId):
        return True
    if(prod1.title == prod2.title):
        return True
    if(prod1.productUrl == prod2.productUrl):
        return True
    #if(pd.notnull(prod1.description) and pd.notnull(prod1.description) and (prod1.description == prod2.description)):
    if(prod1.description == prod2.description):
        return True

    
    return False


In [ ]:
#df3.shape[0]
count = 0
for i in range(0, (int)(df3.shape[0]/1000.0)):
    for j in range(i+1, (int)(df3.shape[0]/1000.0)):
        if(isDuplicate(df3.iloc[i], df3.iloc[j])):
            count +=1
            print(count)
            print("duplicate products:", i,j, df3.iloc[i].productId, df3.iloc[j].productId)